# Introduction to (Integer) Linear Programming

**Please note that all stories/exercises listed in this notebook are examples and do not resemble a real-life scenario!**

In [ ]:
# Installing the relevant packages
!pip install pandas pyomo
!sudo apt-get update
!sudo apt-get install -y -qq coinor-cbc

In [ ]:
import pandas as pd
import pyomo.environ as pyo

In [ ]:
import random
import itertools

In [ ]:
# Use the CBC Solver
solver = pyo.SolverFactory('cbc', executable='/usr/bin/cbc')

### Knapsack problem
![Knapsack Problem](https://github.com/dijkstrar/ns-OR-workshop/blob/bd090ab1b3c5813091868efec13856e661f62b4a/images/knapsack.png?raw=true)

Consider the knapsack problem. For this problem we have a backpack with a size limitation, furthermore we have a set of books, each with their own monetary value and weight. For this problem we aim to take all the books with the largest monetary weight, while not exceeding the backpack's limits... We are not allowed to take fractions of books with us!

A first way to solve this problem would be to iterate over all possible solutions ($2^5=32$). While this is feasible for a small number of books, the problem becomes complicated whenever we have 30 books or more (Then the number of solutions equals: $2^{30}=1,073,741,824$).


A more intelligent way to solve such a problem would be to solve it by means of Integer Linear Programming.

Let's give it a try :)

 A first intuitive approach would be to code every constraint by hand. When doing this, we transform the mathematical problem (displayed in the image below) into Python code, to pass it along to the CBC Solver!

 ![Mathematical Formulation](https://github.com/dijkstrar/ns-OR-workshop/blob/bd090ab1b3c5813091868efec13856e661f62b4a/images/ILP_hand.png?raw=true)

In [ ]:
# Define a model
model = pyo.ConcreteModel('Knapsack Problem (by hand)')

# Declare the Decision Variables
model.y_green = pyo.Var(within=pyo.Binary) 
model.y_blue = pyo.Var(within=pyo.Binary) 
model.y_yellow = pyo.Var(within=pyo.Binary) 
model.y_pink = pyo.Var(within=pyo.Binary) 
model.y_grey = pyo.Var(within=pyo.Binary) 
# When specifying the domain (Binary -> 0/1), we implement the 0/1 constraint


# Declare objective
model.objective = pyo.Objective(expr=4*model.y_green+2*model.y_blue+10*model.y_yellow+3*model.y_pink+2*model.y_grey,
                                                   sense=pyo.maximize)

# Declare maximum weight constraint
model.weight_constraint = pyo.Constraint(expr = 12*model.y_green+2*model.y_blue+4*model.y_yellow+1*model.y_pink+1*model.y_grey <= 15)

# Solve
result = solver.solve(model)

# Print solutions
print(f"Solver status:{result.solver.status}, {result.solver.termination_condition} in {result.solver.time}s")
print(f"Book Green picked: {model.y_green.value}")
print(f"Book Blue picked: {model.y_blue.value}")
print(f"Book Yellow picked: {model.y_yellow.value}")
print(f"Book Pink picked: {model.y_pink.value}")
print(f"Book Grey picked: {model.y_grey.value}")
print(f"Maximizes objective value to: {model.objective()}")

Well done, the solver has found a solution, it is optimal and we were able to find it really fast!

In this example several important things are displayed:
- We specify the domain of decision variables. In this example we used `pyo.Binary` for 0/1 variables, however we may also use `pyo.NonNegativeReals` for real-valued variables (floats), or `pyo.NonNegativeIntegers` for Integer decision variables!
- In order to obtain the value of a specific decision variable, we use the `.value` attribute of the decision variable
- Lastly, the `model.objective()` displays the obtained objective value!

### But...
Imagine 200 extra books get added. Typing all this out by hand would be a lot of work... Therefore we discourage the approach listed above :)

Let's try a more Pythonic way to solve the problem!

In the Python code below, we will be implementing the following mathematical model. Where $I$ denotes the set of books, $P_i$ denotes the monetary value of each book, $w_i$ denotes the weight of each book and $C$ is the capacity of the knapsack

 ![Mathematical Formulation](https://github.com/dijkstrar/ns-OR-workshop/blob/bd090ab1b3c5813091868efec13856e661f62b4a/images/ILP_formal.png?raw=true)

In [ ]:
# First we define the values and weights per book
df=pd.DataFrame()
df.at['green','value']=4
df.at['green','weight']=12
df.at['blue','value']=2
df.at['blue','weight']=2
df.at['yellow','value']=10
df.at['yellow','weight']=4
df.at['pink','value']=3
df.at['pink','weight']=1
df.at['grey','value']=2
df.at['grey','weight']=1
df

In [ ]:
# Define a model
model = pyo.ConcreteModel('Knapsack Problem')

# Declare set of books
model.books = pyo.Set(initialize=df.index)

# Declare the Decision Variables
model.y = pyo.Var(model.books,within=pyo.Binary) 
# When specifying the domain, we implement the 0/1 constraint


# Declare objective
model.objective = pyo.Objective(expr=pyo.quicksum([df['value'][book]*model.y[book] for 
                                                   book in model.books ]),
                                                   sense=pyo.maximize)

# Declare maximum weight constraint
model.weight_constraint = pyo.Constraint(expr = pyo.quicksum([df['weight'][book]*model.y[book] 
                                                  for book in model.books]) <= 15)

# Solve
result = solver.solve(model)

# Print solutions
print(f"Solver status:{result.solver.status}, {result.solver.termination_condition} in {result.solver.time}s")
for book in model.books:
    print(f"Book {book} picked: {model.y[book].value}")
print(f"Maximizes objective value to: {model.objective()}")

Do you notice the difference in lines of code we need to find the solution? By using list comprehensions and other pyomo utilities such as: `pyo.quicksum()`, we made our life easier!

After solving the model, we can dive into the specifics of the solved model.

By calling `model.display()` we obtain all relevant information of the model. 
- We are able to observe the value of decision variables
- We are able to observe the obtained objective value
- We are able to observe the constraints (and whether or not we exceed them...)

In [ ]:
model.display()

When inspecting the Constraints block, we can see that we currently use 8kg in our backpack, while we have a limit of 15kg. We're not using all our capacity.

# Now it is up to you!
After the brief introduction, and code examples that work, you must do the work now!

## Exercise: We are adding new rules!
New rules are introduced...
- Only two books can be picked at most...
- The yellow and blue book cannot be picked at the same time...


As a mathematician, it is up to you to define the new model. To do this, I need you to perform the following!
1. Define the new constraints in mathematical terms!
2. Implement the constraints in the model underneath

In [ ]:
# Define a model
model = pyo.ConcreteModel('Knapsack Problem')

# Declare set of books
model.books = pyo.Set(initialize=df.index)

# Declare the Decision Variables
model.y = pyo.Var(model.books,within=pyo.Binary) 
# When specifying the domain, we implement the 0/1 constraint


# Declare objective
model.objective = pyo.Objective(expr=pyo.quicksum([df['value'][book]*model.y[book] for 
                                                   book in model.books ]),
                                                   sense=pyo.maximize)

# Declare maximum weight constraint
model.weight_constraint = pyo.Constraint(expr = pyo.quicksum([df['weight'][book]*model.y[book] 
                                                  for book in model.books]) <= 15)

# Declare the two books maximum rule
raise ValueError('Two book constraint not implemented! Please implement and remove this line')

# Declare the yellow/blue book rule
raise ValueError('Yellow/blue book rule not implemented! Please implement and remove this line')

# Solve
result = solver.solve(model)

# Print solutions
print(f"Solver status:{result.solver.status}, {result.solver.termination_condition} in {result.solver.time}s")
for book in model.books:
    print(f"Book {book} picked: {model.y[book].value}")
print(f"Maximizes objective value to: {model.objective()}")

### Solution!
First we display the mathematical model to be solved:

 ![Mathematical Formulation](https://github.com/dijkstrar/ns-OR-workshop/blob/bd090ab1b3c5813091868efec13856e661f62b4a/images/ILP_q1.png?raw=true)

The Python code below demonstrates how to transform the mathematical model to a form the solver is able to use!

In [ ]:
# Define a model
model = pyo.ConcreteModel('Knapsack Problem')

# Declare set of books
model.books = pyo.Set(initialize=df.index)

# Declare the Decision Variables
model.y = pyo.Var(model.books,within=pyo.Binary) 
# When specifying the domain, we implement the 0/1 constraint


# Declare objective
model.objective = pyo.Objective(expr=pyo.quicksum([df['value'][book]*model.y[book] for 
                                                   book in model.books ]),
                                                   sense=pyo.maximize)

# Declare maximum weight constraint
model.weight_constraint = pyo.Constraint(expr = pyo.quicksum([df['weight'][book]*model.y[book] 
                                                  for book in model.books]) <= 15)

# Declare the two books maximum rule
model.two_book_constraint = pyo.Constraint(expr = pyo.quicksum([model.y[book] 
                                                  for book in model.books]) <= 2)

# Declare the yellow/blue book rule
model.yellow_blue_book_constraint = pyo.Constraint(expr = model.y['yellow']+model.y['blue'] <= 1)

# Solve
result = solver.solve(model)

# Print solutions
print(f"Solver status:{result.solver.status}, {result.solver.termination_condition} in {result.solver.time}s")
for book in model.books:
    print(f"Book {book} picked: {model.y[book].value}")
print(f"Maximizes objective value to: {model.objective()}")

In the cell above we implemented both constraints!

1. This constraint was implemented with the following logic. If a book gets picked, the decision value is set to 1. If we allow two books to be picked at most, we need to make sure that at most two of the decision variables are set to 1! Therefore we can add the constraint that the sum of the decision variables can be 2 at most.
2. This constraint was implemented with the following logic. If a book gets picked, the decision value is set to 1. If both the blue and yellow book get picked, the sum of their decision variables equals two. This is not allowed by our rule, and thus this has an upperbound of 1!

You can also try experimenting with additional rules, such as:
- We must always choose the green book
- If the yellow book gets picked, we must take the blue book

# A New Problem Formulation: How much ov-bikes and e-bikes to produce?
In a fictive scenario, NS has to produce its ov-bikes on their own. The predicted revenue (for the entire lifetime) per type of bike is displayed below

In [ ]:
df = pd.DataFrame()
df.at['ov-bike','revenue']=1000
df.at['e-bike','revenue']=4000
df

Producing these bikes is not easy, there are several parts needed to produce these bikes. Per bike a different quantity is required. These quantities are displayed below!

In [ ]:
df_parts = pd.DataFrame()
df_parts.at['ov-bike','saddles']=1
df_parts.at['e-bike','saddles']=1
df_parts.at['ov-bike','metal']=15
df_parts.at['e-bike','metal']=50
df_parts.at['ov-bike','chain_length']=3
df_parts.at['e-bike','chain_length']=10
df_parts

For these parts, there is a limited quantity available...

In [ ]:
df_parts_capacity = pd.DataFrame()
df_parts_capacity.at['saddles','capacity']=3000
df_parts_capacity.at['metal','capacity']=13000
df_parts_capacity.at['chain_length','capacity']=300
df_parts_capacity

Additionally, a maximum of 15 e-bikes can be produced!

As a project manager it is up to you to decide how much bikes to produce!

1. Define the model in mathematical terms
2. Implement it in pyomo :)
    - For this problem we want to produce more than 1 bike, therefore the domain of the decision variable must be altered. Please use: `pyo.NonNegativeIntegers` to produce an integer amount of bikes.
    - If you for some reason want to produce fractional bikes, go ahead and use `pyo.NonNegativeReals` 

In [ ]:
limit_e_bikes = 15

In [ ]:
# Define a model
model = pyo.ConcreteModel('ov-bike Problem')

model.bikes = pyo.Set(initialize=df.index)
model.parts = pyo.Set(initialize=df_parts_capacity.index)

# Declare the Decision Variables
model.produced = pyo.Var(model.bikes,within=pyo.NonNegativeIntegers)

# Construct the model!
raise ValueError('Model not implemented... Remove once done!')


# Keep this code
# Solve
result = solver.solve(model)

# Print solutions
print(f"Solver status:{result.solver.status}, {result.solver.termination_condition} in {result.solver.time}s")
for bike in model.bikes:
    print(f"Bike {bike} produced: {model.produced[bike].value}")
print(f"Maximizes objective value to: {model.objective()}")

### Solution

First we start writing the mathematical notation, displayed below.
For this formulation, $B$ denotes the set of bikes we can produce (ov-bike and e-bike). $y_b$ denotes the quantity of bike $b$ we can produce. Lastly $q_{(b,\text{saddles})}$ denotes the quantity of saddles required for producing bike $b$. Likewise variables exist for the metal and chain length required per bike. Moreover, $R_b$ describes the revenue obtained for a bike of type $b$!

Note how we are not allowed to produce fractional bikes, requiring our decision variable $y_b$ to be positive-Integer ($\mathbb{N^+}$), as negative production is not allowed!

 ![Mathematical Formulation](https://github.com/dijkstrar/ns-OR-workshop/blob/bd090ab1b3c5813091868efec13856e661f62b4a/images/ILP_ovbike_nolambda.png?raw=true)

In [ ]:
# Define a model
model = pyo.ConcreteModel('ov-bike Problem')

# Declare set of bikes and parts
model.bikes = pyo.Set(initialize=df.index)
model.parts = pyo.Set(initialize=df_parts_capacity.index)

# Declare the Decision Variables
model.produced = pyo.Var(model.bikes,within=pyo.NonNegativeIntegers) 
# We do not allow fractional bikes to be produced.


# Declare objective
model.objective = pyo.Objective(expr=pyo.quicksum([df['revenue'][bike]*model.produced[bike] for 
                                                   bike in model.bikes ]),
                                                   sense=pyo.maximize)

# Declare constraints per part
model.maximum_saddles = pyo.Constraint(expr = pyo.quicksum([df_parts['saddles'][bike]*model.produced[bike] 
                                                  for bike in model.bikes]) <= df_parts_capacity['capacity']['saddles'])

model.maximum_metal = pyo.Constraint(expr = pyo.quicksum([df_parts['metal'][bike]*model.produced[bike] 
                                                  for bike in model.bikes]) <= df_parts_capacity['capacity']['metal'])

model.maximum_chain = pyo.Constraint(expr = pyo.quicksum([df_parts['chain_length'][bike]*model.produced[bike] 
                                                  for bike in model.bikes]) <= df_parts_capacity['capacity']['chain_length'])
# Do not produce more than 15 e-bikes
model.limit_e_bikes = pyo.Constraint(expr = model.produced['e-bike'] <= limit_e_bikes)


# Solve
result = solver.solve(model)

# Print solutions
print(f"Solver status:{result.solver.status}, {result.solver.termination_condition} in {result.solver.time}s")
for bike in model.bikes:
    print(f"Bike {bike} produced: {model.produced[bike].value}")
print(f"Maximizes objective value to: {model.objective()}")

Can you see we're repeating ourselves?
For each part, we're writing the same constraint, but altering the name of the considered part.

Let us introduce a new way of writing the constraint in order to prevent this from happening!
In the model formulation below, see how we have written the second constraint, utilising all the parts we are considering?


For this we are introducing a new set $P$, which consists of all the parts needed for producing the bikes (saddles, metal and chains), with a total capacity of $C_p$ for each part. Moreover, we introduce $q_{bp}$ which denotes the quantity of material $p$ needed for producing bike $b$. This allows us to write a very neat model.



 ![Mathematical Formulation](https://github.com/dijkstrar/ns-OR-workshop/blob/bd090ab1b3c5813091868efec13856e661f62b4a/images/ILP_ovbike_lambda.png?raw=true)


This new constraint we can easily transform into a more Pythonic way in Pyomo. Let us introduce the `lambda` operator!

With the `lambda` operator, we are able to write constraints in an iterative way. This allows us to write a constraint for each `part` in `model.parts` in a single line of code.

For this we must use the line of code below,

```
model.maximum_part_usage = pyo.Constraint(model.parts, rule=lambda model, part:
                                            pyo.quicksum([df_parts[part][bike]*model.produced[bike] 
                                            for bike in model.bikes]) <= df_parts_capacity['capacity'][part])
```
 where the following happens.
- We consider the `model.parts` and start iterating with a rule. This rule describes what must happen in each iteration. We can use this rule to formulate a constraint per iteration.
- While iterating, we are using the `model` and each `part` in `model.parts`.
- The variable `part` takes on the values as described in `model.parts` and we can thus use it in our constraint.

This allows us to write three constraints into a single line of code.

Note that this is only possible if we start repeating constraints, while only changing a single value in our constraint!

When implementing it, it would look like this!

In [ ]:
# Define a model
model = pyo.ConcreteModel('ov-bike Problem')

# Declare set of bikes and parts
model.bikes = pyo.Set(initialize=df.index)
model.parts = pyo.Set(initialize=df_parts_capacity.index)

# Declare the Decision Variables
model.produced = pyo.Var(model.bikes,within=pyo.NonNegativeIntegers) 
# We do not allow fractional bikes to be produced.


# Declare objective
model.objective = pyo.Objective(expr=pyo.quicksum([df['revenue'][bike]*model.produced[bike] for 
                                                   bike in model.bikes ]),
                                                   sense=pyo.maximize)

# Declare constraints per part
model.maximum_part_usage = pyo.Constraint(model.parts, rule=lambda model, part:
                                            pyo.quicksum([df_parts[part][bike]*model.produced[bike] 
                                            for bike in model.bikes]) <= df_parts_capacity['capacity'][part])
# Do not produce more than 15 e-bikes
model.limit_e_bikes = pyo.Constraint(expr = model.produced['e-bike'] <= 15)


# Solve
result = solver.solve(model)

# Print solutions
print(f"Solver status:{result.solver.status}, {result.solver.termination_condition} in {result.solver.time}s")
for bike in model.bikes:
    print(f"Bike {bike} produced: {model.produced[bike].value}")
print(f"Maximizes objective value to: {model.objective()}")

See how this is shorter?!

### Question
Based on the model output below, which constraints are troublesome for the project manager? Which of these constraints must be addressed immediately to obtain more revenue?

In [ ]:
model.display()

### Answer
There are two constraints which are very troublesome.
1. First being the limited number of e-bikes. We can observe this by inspecting the obtained constraint value by the model ( inspect the `Body` part of the `limit_e_bikes` constraint, namely 15). This indicates that the upper-limit of 15 e-bikes is reached!
2. Second, the total chain length is also troublesome. In the same reasoning we can see that we have depleted this entire resource.

It may be useful for the bike producer to increase both these limits!


**Try to alter the values of these capacities and try to understand what happens!**


### Question
Assume that the Research and Development costs of the e-bike production play an important role on whether or not to produce e-bikes! This implies that if we produce e-bikes, a fixed cost is incurred for the R&D process.

This makes decision making more complicated....

NS has asked your help on whether or not to indulge in the production of these e-bikes!
The estimated research and development costs equal: **30,000**, which is a lot of money!

In order to aid in this decision, NS needs you to do the following:
1. Define the new model in mathematical terms
2. Implement it in Pyomo and solve it!

What would you advise NS to do?

In [ ]:
rd_costs = 30000

In [ ]:
df_parts_capacity = pd.DataFrame()
df_parts_capacity.at['saddles','capacity']=3000
df_parts_capacity.at['metal','capacity']=13000
df_parts_capacity.at['chain_length','capacity']=300

df_parts = pd.DataFrame()
df_parts.at['ov-bike','saddles']=1
df_parts.at['e-bike','saddles']=1
df_parts.at['ov-bike','metal']=15
df_parts.at['e-bike','metal']=50
df_parts.at['ov-bike','chain_length']=3
df_parts.at['e-bike','chain_length']=10


In [ ]:
# Define a model
model = pyo.ConcreteModel('ov-bike Problem')

# Declare set of bikes and parts
model.bikes = pyo.Set(initialize=df.index)
model.parts = pyo.Set(initialize=df_parts_capacity.index)

# Declare the Decision Variables
model.produced = pyo.Var(model.bikes,within=pyo.NonNegativeIntegers) 
# We do not allow fractional bikes to be produced.
raise ValueError('Add a new decision variable, and alter one of the constraints to account for the R&D costs.')


# Declare objective
model.objective = pyo.Objective(expr=pyo.quicksum([df['revenue'][bike]*model.produced[bike] for 
                                                   bike in model.bikes ]),
                                                   sense=pyo.maximize)

# Declare constraints per part
model.maximum_part_usage = pyo.Constraint(model.parts, rule=lambda model, part:
                                            pyo.quicksum([df_parts[part][bike]*model.produced[bike] 
                                            for bike in model.bikes]) <= df_parts_capacity['capacity'][part])
# Do not produce more than 15 e-bikes
model.limit_e_bikes = pyo.Constraint(expr = model.produced['e-bike'] <= limit_e_bikes)


# Solve
result = solver.solve(model)

# Print solutions
print(f"Solver status:{result.solver.status}, {result.solver.termination_condition} in {result.solver.time}s")
for bike in model.bikes:
    print(f"Bike {bike} produced: {model.produced[bike].value}")
print(f"Maximizes objective value to: {model.objective()}")

##### Hints
A new decision variable needs to be introduced, namely a binary variable on whether or not to produce the e-bike (say $x$).
If do not incur the R&D costs, we cannot produce any e-bikes. This implies that a relationship exists between two decision variables (production of e-bikes, and the binary variable indicating whether we incur R&D costs.)

if $x=0$: production of e-bikes equals 0

if $x=1$: production of e-bikes can be 15 at most!

### Solution

This question is a more difficult question. It requires some intuition on how to decision variables may interact with eachother.
In the image below we have displayed the mathematical formulation. In this formulation, we have introduced a new decision variable $x$, which denotes whether or not we incur the R&D costs. The variable $D$ denotes the costs of R&D (30,000)!

For this formulation we may use the trick that whenever $x=0$ we cannot produce any e-bikes, and we thus require $y_{\text{e-bike}}\leq 0*15$. However, if we are allowed to produce e-bikes, $x=1$, we can produce a limited quantity of 15, yielding $y_{\text{e-bike}}\leq 1*15$ as a constraint. With this logic in mind, we can force the model to make a choice on the value of $x$ by including the effect of $x$ in the objective function, namely a decrease in revenue of 30,000 if we decide to incur the R&D costs.

We thus let the solver decide on the value of $x$ by incorporating it in the constraint **and** objective value!

![Mathematical Formulation](https://github.com/dijkstrar/ns-OR-workshop/blob/bd090ab1b3c5813091868efec13856e661f62b4a/images/ILP_ovbike_RDcosts.png?raw=true)

In [ ]:
# Define a model
model = pyo.ConcreteModel('ov-bike Problem')

# Declare set of bikes and parts
model.bikes = pyo.Set(initialize=df.index)
model.parts = pyo.Set(initialize=df_parts_capacity.index)

# Declare the Decision Variables
model.produced = pyo.Var(model.bikes,within=pyo.NonNegativeIntegers) 
# We do not allow fractional bikes to be produced.
model.incur_rd_costs = pyo.Var(within=pyo.Binary) 


# Declare objective
model.objective = pyo.Objective(expr=pyo.quicksum([df['revenue'][bike]*model.produced[bike] for 
                                                   bike in model.bikes ]) - rd_costs*model.incur_rd_costs,
                                                   sense=pyo.maximize)

# Declare constraints per part
model.maximum_part_usage = pyo.Constraint(model.parts, rule=lambda model, part:
                                            pyo.quicksum([df_parts[part][bike]*model.produced[bike] 
                                            for bike in model.bikes]) <= df_parts_capacity['capacity'][part])
# Do not produce more than 15 e-bikes
model.limit_e_bikes = pyo.Constraint(expr = model.produced['e-bike'] <= limit_e_bikes*model.incur_rd_costs)


# Solve
result = solver.solve(model)

# Print solutions
print(f"Solver status:{result.solver.status}, {result.solver.termination_condition} in {result.solver.time}s")
for bike in model.bikes:
    print(f"Bike {bike} produced: {model.produced[bike].value}")
print(f"Maximizes objective value to: {model.objective()}")

See how the solver has not decided to produce any e-bikes. Apparently it is not beneficial to incur the R&D costs!

This changes the choices we make, namely we do produce more ov-bikes, resulting in a revenue decrease of 10,000

# A New Problem Formulation: Help my Product Owner and Scrum master to plan their sprint!
For the new sprint, the product owner and scrum master of your team have asked for your help! Your team consists of 12 members and there is a set of work to be done. 

For the tasks of work, we have defined the following
1. A Happiness Rating for the employee to perform this task
2. The projected amount of effort needed for the employee to complete the task (think of this like the outcome of pokering user stories).

Furthermore, each employee has an upper limit of how much work they can do this sprint (working part-time, holidays etc.)

For this problem, please provide the scrum master with a sprint planning, highlighting the assignment of user stories to employees. The focus is to maximise employee-happiness and not exceed the availability of the employee! A task can only be assigned to one employee, and cannot be done by multiple employees at the same time!

For this, we have defined the following variables

In [ ]:
number_employees = 8
number_tasks = 15

In [ ]:
tasks = [f"task_{i:02}" for i in range(1,number_tasks+1)]
tasks

Each task requires a set amount effort for the task to be completed (story points after pokering)

In [ ]:
random.seed(42)
data = {
    "Effort": [random.randint(3, 8) for _ in range(len(tasks))]
}
df_effort = pd.DataFrame(data, index=tasks)
df_effort

Let's define the employees

In [ ]:
employees = [f"employee_{chr(65 + i)}" for i in range(number_employees)]
employees

We define for each employee how much availability there is!

In [ ]:
random.seed(42)
data = {
    "Availability": [random.randint(7, 15) for _ in range(len(employees))]
}

df_availability = pd.DataFrame(data, index=employees)
df_availability

For each employee, we indicate how much happiness the employee obtains from performing this task

In [ ]:
random.seed(42)
data = {emp: [random.randint(1, 10) for _ in range(len(tasks))] for emp in employees}
df_happiness = pd.DataFrame(data, index=tasks)
df_happiness

It is up to you as the mathematician to provide the product owner and scrum master with a planning of this sprint!

**Remember that a task can only be assigned to one person!!**

Please do the following:
1. Write the mathematical version of this model.
2. Implement it in Pyomo (and try to do it as efficiently as possible using `lambda` operators!)

In [ ]:
# Define a model
model = pyo.ConcreteModel('Sprint Planning Problem')

# Declare set of employees and tasks
model.tasks = pyo.Set(initialize=tasks)
model.employees = pyo.Set(initialize=employees)

# Declare the Decision Variables
model.assignment = pyo.Var(model.employees, model.tasks ,within=pyo.Binary)
# Read this as: task abc is assigned to employee xyz can be 0/1. 
# in order to access this variable, please use: model.assignment[employee, task]!
# proceed as usual! 

# Declare objective
model.objective = pyo.Objective(expr=pyo.quicksum([df_happiness[employee][task]*model.assignment[employee, task] for 
                                                   employee in model.employees for task in model.tasks ]),
                                                   sense=pyo.maximize)

# This is a new method of implementing the constraint:
# For every task, we want to ensure that it is only assigned once. Within one line, we can define multiple constraints at once. 
# One constraint per task!
model.ensure_single_assignment = pyo.Constraint(model.tasks, rule=lambda model, task:
                                                       pyo.quicksum([model.assignment[employee, task] 
                                                  for employee in model.employees]) <= 1)

raise ValueError('Please implement the constraint for respecting the availability of each employee! Please try to formulate the constraint in the same way as above!')

# Solve
result = solver.solve(model)

# Print solutions
print(f"Solver status:{result.solver.status}, {result.solver.termination_condition} in {result.solver.time}s")
for (employee, task) in list(itertools.product(employees, tasks)):
    if model.assignment[employee, task].value:
        print(f"Task {task} Assigned to {employee}")
print(f"Maximizes objective value to: {model.objective()}")

### Solution

To start this exercise, we need to create multiple sets, namely set $E$ for all the employees and $T$ for all the tasks. We create $C_e$ which denotes the availability for each employee $e$, and define $f_t$ which denotes the required amount of work to complete task $t$.
Lastly, $h_{et}$ denotes the happiness level of employee $e$ performing task $t$.


As we need to assign employees to decision variables, we can create a decision variable for assigning an employee $e$ to task $t$. This requires us to define a decision variable with multiple indices. We create a decision variable for each employee+task combination (120 decision variables in our case). This decision variable is called $y_{et}$ and describes whether employee $e$ is assigned to task $t$. Since the employee must complete the task, the domain of the decision variable is Binary.

With this provided information, we may write the following model:

![Mathematical Formulation](https://github.com/dijkstrar/ns-OR-workshop/blob/bd090ab1b3c5813091868efec13856e661f62b4a/images/ILP_sprint_planning.png?raw=true)

For this model, we define two constraints:
1. The availability of each employee is respected. That is, for each employee the sum of effort of the assigned tasks does not exceed the employee's capacity
2. Second, each task can be assigned to one employee at most. Therefore, for each task we need to make sure that when iterating over all the employees, at most one employee is assigned to this task. 

In [ ]:
# Define a model
model = pyo.ConcreteModel('Sprint Planning Problem')

# Declare set of employees and tasks
model.tasks = pyo.Set(initialize=tasks)
model.employees = pyo.Set(initialize=employees)

# Declare the Decision Variables
model.assignment = pyo.Var(model.employees, model.tasks ,within=pyo.Binary)
# Read this as: task abc is assigned to employee xyz can be 0/1. 
# in order to access this variable, please use: model.assignment[employee, task]
# proceed as usual! 

# Declare objective
model.objective = pyo.Objective(expr=pyo.quicksum([df_happiness[employee][task]*model.assignment[employee, task] for 
                                                   employee in model.employees for task in model.tasks ]),
                                                   sense=pyo.maximize)

# Declare constraints per part
model.constrain_employee_availability = pyo.Constraint(model.employees, rule=lambda model, employee:
                                                       pyo.quicksum([df_effort['Effort'][task]*model.assignment[employee, task] 
                                                  for task in model.tasks]) <= df_availability['Availability'][employee])



model.ensure_single_assignment = pyo.Constraint(model.tasks, rule=lambda model, task:
                                                       pyo.quicksum([model.assignment[employee, task] 
                                                  for employee in model.employees]) <= 1)


# Solve
result = solver.solve(model)

# Print solutions
print(f"Solver status:{result.solver.status}, {result.solver.termination_condition} in {result.solver.time}s")
for (employee, task) in list(itertools.product(employees, tasks)):
    if model.assignment[employee, task].value:
        print(f"Task {task} Assigned to {employee}")
print(f"Maximizes objective value to: {model.objective()}")


In [ ]:
for employee in employees:
    print(f"{employee} has availability: {df_availability['Availability'][employee]} and is assigned: {sum([df_effort['Effort'][task]*model.assignment[employee, task].value for task in model.tasks])}")

As an additional exercise, you can think of extra constraints to implement.
1. task_04 is currently not assigned to any employee, please ensure that this task is assigned to any employee.
2. A specific employee F currently has two tasks assigned, however due to his/her inability to switch contexts, this employee prefers to only have one task assigned...

Try to see if you can implement these constraints

# The Next Codeblock will be an example discussed during the workshop at the end! Feel free to solve it on your own and think along :)

# Minimise the duration of both projects!
NS has two projects to work on simultaneously and it aims to finish both projects as soon as possible. If NS does nothing project 1: "Nighttrain Alaska" will be finished after $200$ days and project 2: "Nighttrain Vladivostok" after $100$ days. 

However, NS can allocate extra money to the projects. For every €10.000 it spends on project 1, the project finishes $3$ days earlier. If the company spends €10.000 on project 2, the second project will finish $2$ days earlier. The total budget of NS is €500,000. Aiming to complete both projects as soon as possible, the objective is to **minimize the maximum completion** time of the two projects. Formulate the problem above as an integer linear optimization problem and solve it with *Pyomo*.

1. Define the mathematical formulation to solve this problem!
2. Implement it in Pyomo and solve it!


In [ ]:
budget = 50 # in thousands!
default_finish_time = {'p1':200,'p2':100}
decrement={'p1':3,'p2':2}

In [ ]:
model = pyo.ConcreteModel('Nighttrain Project Management')

# Declare decision variables
model.projects = pyo.Set(initialize=default_finish_time.keys())

# This decision variable indicates how often we spend the 10k to decrease the project time.
# It is defined for both projects!
model.ten_k_spent_per_project = pyo.Var(model.projects ,within=pyo.NonNegativeIntegers)
raise ValueError('Please implement the rest of this model!')


# Keep this!
result = solver.solve(model)
model.display()


What should NS do?

### Hints
At first hand you might think the objective function is non-linear, as $\min$ or $\max$ functions do behave in a non-linear way..

Though this is true, in this special case ($\min \max$ problem) is possible to rewrite the objective function in a linear way.

In order to do this, we need you to do the following
1. Define two decision variables, which indicate the projected end-time for each project.
    - These variables depend on how much is invested in each project!
2. We need you to define a variable, which will indicate the maximum of the two projected end-times. This variable must be minimized.
    - If we do not implement any additional constraints, this maximum duration variable has no dependency with other variables, and will thus automatically be set to 0 (as it is a minmization problem!)
3. Therefore we need to enforce that the maximum duration variable will always be bigger than both the projected end-dates for both projects.
    - Our minimization function will try to minimize the variable defined in Item 2; and the constraints will make sure that a relation exists between the projected end-time and maximum project duration variable!
    

# Solution

In order to create this solution, we first need to create the necessary variables.
First we create $P$ which is the set of projects. Second, we create $f_p$ which is the default finish time of project $p$ if we do not spend any money on the project. Third, we define $d_p$, which is the decrease in duration for project $p$ per 10,000 spent on it.

For our decision variable, we define how much we spend the 10,000 on a specific project $p$, this is our decision variable $x_p$. As we can spend multiple 10,000's on a project, the domain of this decision variable is Integer ($\mathbb{N^+}$). As we cannot spend more than 500,000; we need to impose a constraint such that the sum of the 10,000's spent, does not exceed 50 (10,000*50=500,000). 

Lastly, for our objective formulation, we want to minimise the maximum completion time of both the projects. The completion time of project $p$ is defined as: $f_{p}-d_px_{p}$. 

The maximum project duration is then defined as: $\max_{p\in P} (f_{p}-d_px_{p})$.
When attempting to minimise this, we can sense that minimisation of this function is hard, as it seems this function behaves in a non-linear way.

This yields us with the initial model formulation presented in the image below:

![Mathematical Formulation](https://github.com/dijkstrar/ns-OR-workshop/blob/bd090ab1b3c5813091868efec13856e661f62b4a/images/ILP_nighttrain_min_max.png?raw=true)

As our initial function seems to be non-linear, and we require linear constraints and objectuve functions, we have a problem...

This requires us to apply a new trick; namely to rewrite the objective function and create new constraints and decision variables.
1. We create a new decision variable $z$ which denotes the maximum completion time of both projects.
2. We need to create a constraint where we correctly set the value of $z$, namely $z$ being bigger than $f_{p}-d_{p}x_{p} \forall p \in P$.
3. Lastly, as we try to minimize the maximum completion time ($z$), we must alter the objective function, namely to minimize $z$.

This yields us with the fomulation displayed below:


![Mathematical Formulation](https://github.com/dijkstrar/ns-OR-workshop/blob/bd090ab1b3c5813091868efec13856e661f62b4a/images/ILP_nighttrain_min_max_solution.png?raw=true)

In [ ]:
model = pyo.ConcreteModel('Nighttrain Project Management')

# Declare decision variables
model.projects = pyo.Set(initialize=default_finish_time.keys())

model.ten_k_spent_per_project = pyo.Var(model.projects ,within=pyo.NonNegativeIntegers)
model.maximum_project_duration = pyo.Var(within=pyo.NonNegativeIntegers)

model.objective = pyo.Objective(expr=model.maximum_project_duration
                                ,sense=pyo.minimize)

model.determine_max_duration = pyo.Constraint(model.projects , rule=lambda model, project:
                              model.maximum_project_duration>=default_finish_time[project]-
                              model.ten_k_spent_per_project[project]*decrement[project])

model.budget = pyo.Constraint(expr=pyo.quicksum([model.ten_k_spent_per_project[project] for 
                                                 project in model.projects])<=budget)

result = solver.solve(model)
model.display()

In [ ]:
result.solver.time

#### What should NS do?
From the model output above, a solution has been found in 0.007 seconds

This solution dictates the following points:
1. We need to spend 400,000 euros to decrease the duration of Project 1.
2. We need to spend 100,000 euros to decrease the duration of Project 2.
3. We deplete our entire budget...
4. But we finish within 80 days! :)

# The End

I hoped you liked this notebook and that you learnt some new tricks! As a result of attending this workshop, I hope you try to incorporate the technique of Integer Linear Programming in your daily work!

Can you think of any interesting usecases?